In [1]:
from array import array
from skmultiflow.data import RegressionGenerator
from sail.models.torch.rnn import RNNRegressor
import numpy as np

/Users/seshu/opt/anaconda3/envs/sail_test/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_samples = 6000

stream = RegressionGenerator(random_state=1,
                             n_samples=n_samples,
                             n_features=12)
learner_gru = RNNRegressor(input_units=12, output_units=1, hidden_units=20,
                           n_hidden_layers=3, lr=0.001, cell_type="GRU")



In [3]:
cnt = 0
y_true = array('d')
y_pred = array('d')
index = []
i = 0
wait_samples = 10
while cnt < n_samples and stream.has_more_samples():
    X, y = stream.next_sample(batch_size=wait_samples)
    y = np.array(y)
    y = y.reshape(y.shape[0], -1)
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    # Test every n samples
    if (cnt % wait_samples == 0) & (cnt != 0):
        y_true.append(y[0])
        y_pred1 = learner_gru.predict(X)[0][0]
        y_pred.append(y_pred1)
        index.append(i)
        i = i + 1
    learner_gru.partial_fit(X, y)
    learner_gru.predict(X)
    cnt += 1



  epoch    train_loss     dur
-------  ------------  ------
      1    48883.3945  0.0382
      2    34301.1133  0.0016
      3    27334.3008  0.0015
      4    65703.9766  0.0014
      5    13493.4980  0.0013
      6    11087.9775  0.0015
      7    33133.3125  0.0015
      8    22704.7695  0.0020
      9    12029.2207  0.0016
     10    10574.0215  0.0016
     11    16022.1768  0.0021
     12    14969.1172  0.0018
     13    40982.8789  0.0022
     14    14541.8066  0.0019
     15    15495.9297  0.0021
     16    24262.8691  0.0022
     17    39107.3477  0.0015
     18    20577.4004  0.0014
     19    20746.9219  0.0023
     20    31165.3125  0.0033
     21    23119.0176  0.0033
     22    19831.7129  0.0035
     23    18310.6387  0.0031
     24    14857.6846  0.0029
     25    28153.3691  0.0032
     26     9505.2812  0.0027
     27    19152.5078  0.0022
     28    39545.8047  0.0026
     29    45333.3867  0.0021
     30    15265.6816  0.0023
     31    32621.2559  0.0018
     32   

In [5]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'iframe'

y_pred = y_pred.tolist()
y_true = y_true.tolist()
fig = go.Figure()

fig.add_trace(go.Scatter(x=index, y=y_pred, name='predicted',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=index, y=y_true, name='actual',
                         line=dict(color='blue', width=2)))
fig.show()


In [6]:
import torch
import torch.nn as nn
from skorch.regressor import NeuralNetRegressor
from skorch.classifier import NeuralNetClassifier


class _RNNModel(nn.Module):
    """Basic RNN/GRU model.
    Args:
        input_units (int): Number of input units
        output_units (int): Number of output units
        hidden_units (int): Number of hidden units
        n_hidden_layers (int): Number of hidden layers
        output_nonlin (torch.nn.Module instance or None (default=nn.Linear)):
            Non-linearity to apply after last layer, if any.
        dropout (float): Dropout
        squeeze_output (bool): default=False
            Whether to squeeze output. For Skorch consistency.
            https://github.com/skorch-dev/skorch/blob/master/skorch/toy.py
        cell_type (string): default="RNN"
    """
    def __init__(self, input_units, output_units, hidden_units,
                 n_hidden_layers=1, dropout=0.2, output_nonlin = nn.Linear, squeeze_output=False,
                 cell_type="RNN"):
        super(_RNNModel, self).__init__()
        self.input_units = input_units
        self.output_units = output_units
        self.hidden_units = hidden_units
        self.n_hidden_layers = n_hidden_layers
        self.output_nonlin = output_nonlin
        self.squeeze_output = squeeze_output
        if cell_type == "RNN":
            rnn = nn.RNNCell
        elif cell_type == "GRU":
            rnn = nn.GRUCell
        else:
            raise ValueError(f"RNN type {cell_type} is not supported. supported: [RNN, GRU]")

        self.rnns = nn.ModuleList(
            [rnn(self.input_units, self.hidden_units)] +
            [rnn(self.hidden_units, self.hidden_units) for i in range(self.n_hidden_layers - 1)])

        if self.output_nonlin:
            self.out = self.output_nonlin(self.hidden_units, self.output_units)
        self.do = nn.Dropout(p=dropout)
        self.actfn = nn.Tanh()
        self.device = torch.device('cpu')
        self.dtype = torch.float

    def forward(self, x, h0=None, train=False):

        hs = x  # initiate hidden state
        if h0 is None:
            h = torch.zeros(hs.shape[0], self.hidden_units)
            c = torch.zeros(hs.shape[0], self.hidden_units)
        else:
            (h, c) = h0

        # RNN cells
        for i in range(self.n_hidden_layers):
            h = self.rnns[i](hs, h)
            if train:
                hs = self.do(h)
            else:
                hs = h
        y = self.out(hs)
        # return y, (h,c)

        return y


class RNNRegressor1(NeuralNetRegressor):
    """Basic RNN/LSTM/GRU model.
    Args:
        input_units (int): Number of input units
        output_units (int): Number of output units
        hidden_units (int): Number of hidden units
        n_hidden_layers (int): Number of hidden layers
        output_nonlin (torch.nn.Module instance or None (default=nn.Linear)):
            Non-linearity to apply after last layer, if any.
        dropout (float): Dropout
        squeeze_output (bool): default=False
            Whether to squeeze output. Skorch requirement.
        cell_type (string): default="RNN"
        **kwargs: Arbitrary keyword arguments
    """
    def __init__(self, input_units, output_units, hidden_units,
                 n_hidden_layers=1, dropout=0.2, output_nonlin = nn.Linear, squeeze_output=False,
                 cell_type="RNN", **kwargs):
        super(RNNRegressor1, self).__init__(
            module=_RNNModel,
            module__input_units=input_units,
            module__output_units=output_units,
            module__hidden_units=hidden_units,
            module__n_hidden_layers=n_hidden_layers,
            module__dropout=dropout,
            module__output_nonlin=output_nonlin,
            module__squeeze_output=squeeze_output,
            module__cell_type=cell_type,
            train_split=None, batch_size=20,
            **kwargs)

In [7]:
n_samples = 6000

stream = RegressionGenerator(random_state=1,
                             n_samples=n_samples,
                             n_features=12)
learner_gru = RNNRegressor1(input_units=12, output_units=1, hidden_units=20,
                           n_hidden_layers=3, lr=0.001, cell_type="GRU", max_epochs=3)



In [8]:
cnt = 0
y_true = array('d')
y_pred = array('d')
index = []
i = 0
wait_samples = 10
while cnt < n_samples and stream.has_more_samples():
    X, y = stream.next_sample(batch_size=wait_samples)
    y = np.array(y)
    y = y.reshape(y.shape[0], -1)
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    # Test every n samples
    if (cnt % wait_samples == 0) & (cnt != 0):
        y_true.append(y[0])
        y_pred1 = learner_gru.predict(X)[0][0]
        y_pred.append(y_pred1)
        index.append(i)
        i = i + 1
    learner_gru.partial_fit(X, y)
    learner_gru.predict(X)
    cnt += 1

  epoch    train_loss     dur
-------  ------------  ------
      1    48898.6328  0.0038
      2    48862.0391  0.0032
      3    48823.8516  0.0040
      4    34250.8828  0.0020
      5    34203.1211  0.0034
      6    34148.5234  0.0030
      7    27470.8379  0.0018
      8    27390.3945  0.0025
      9    27307.9727  0.0033
     10    65540.2031  0.0018
     11    65359.2695  0.0028
     12    65020.2617  0.0031
     13    13187.8799  0.0018
     14    13031.0557  0.0030
     15    12865.4160  0.0024
     16    10467.0840  0.0023
     17    10188.9492  0.0027
     18     9869.8682  0.0034
     19    31275.2188  0.0017
     20    30489.2129  0.0023
     21    29830.2031  0.0027
     22    19338.7305  0.0022
     23    18577.7168  0.0030
     24    17876.4023  0.0025
     25     9671.4707  0.0017
     26     7683.0884  0.0026
     27     7127.7993  0.0025
     28     7174.8076  0.0017
     29     6732.1655  0.0027
     30     6270.7275  0.0031
     31    11805.3457  0.0016
     32   

In [9]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'iframe'
y_pred = y_pred.tolist()
y_true = y_true.tolist()

fig = go.Figure()

fig.add_trace(go.Scatter(x=index, y=y_pred, name='predicted',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=index, y=y_true, name='actual',
                         line=dict(color='blue', width=2)))
fig.show()